# 09_流式输出和实时交互

## 学习目标
- 理解LangGraph的流式输出机制
- 掌握实时状态更新和事件处理
- 学习异步执行和事件监听
- 实现WebSocket实时通信系统

## 核心概念

### 流式输出
LangGraph支持流式输出，允许我们实时获取执行过程中的中间结果和状态变化。

### 事件系统
通过事件系统，我们可以监听图执行过程中的各种事件，如节点开始、结束、错误等。

### 异步执行
支持异步执行模式，提高系统并发性能和用户体验。

## 环境设置

In [ ]:
import asyncio
import json
import time
from datetime import datetime
from typing import Dict, Any, List, AsyncGenerator
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.state import CompiledStateGraph
from langgraph.checkpoint.memory import MemorySaver
import logging

# 设置日志
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 1. 基础流式输出

### 1.1 简单流式执行

In [ ]:
from typing import TypedDict

class StreamingState(TypedDict):
    messages: List[str]
    current_step: str
    progress: int
    results: Dict[str, Any]

def data_processing_node(state: StreamingState) -> StreamingState:
    """数据处理节点"""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] 开始数据处理...")
    
    # 模拟数据处理过程
    for i in range(1, 6):
        time.sleep(0.5)  # 模拟处理时间
        state["progress"] = i * 20
        state["current_step"] = f"处理数据批次 {i}/5"
        print(f"[{datetime.now().strftime('%H:%M:%S')}] {state['current_step']} - 进度: {state['progress']}%")
    
    state["messages"].append("数据处理完成")
    state["results"]["processed_data"] = [f"数据_{i}" for i in range(1, 101)]
    return state

def analysis_node(state: StreamingState) -> StreamingState:
    """分析节点"""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] 开始数据分析...")
    
    state["current_step"] = "分析数据模式"
    time.sleep(1)
    print(f"[{datetime.now().strftime('%H:%M:%S')}] {state['current_step']}")
    
    state["current_step"] = "生成分析报告"
    time.sleep(1)
    print(f"[{datetime.now().strftime('%H:%M:%S')}] {state['current_step']}")
    
    state["messages"].append("数据分析完成")
    state["results"]["analysis"] = {"pattern": "线性增长", "accuracy": 0.95}
    state["progress"] = 100
    return state

# 创建流式处理图
workflow = StateGraph(StreamingState)
workflow.add_node("process", data_processing_node)
workflow.add_node("analyze", analysis_node)

workflow.add_edge(START, "process")
workflow.add_edge("process", "analyze")
workflow.add_edge("analyze", END)

streaming_app = workflow.compile()

# 执行流式处理
initial_state = {
    "messages": [],
    "current_step": "初始化",
    "progress": 0,
    "results": {}
}

print("=== 开始流式处理 ===")
final_state = streaming_app.invoke(initial_state)
print("\n=== 处理完成 ===")
print(f"最终消息: {final_state['messages']}")
print(f"最终结果: {final_state['results']}")

### 1.2 使用stream()方法进行流式输出

In [ ]:
# 重新定义带有更多输出的节点
def verbose_processing_node(state: StreamingState) -> StreamingState:
    """详细输出的处理节点"""
    state["current_step"] = "开始处理"
    
    for i in range(1, 4):
        time.sleep(0.3)
        state["progress"] = i * 25
        state["current_step"] = f"处理阶段 {i}"
        state["messages"].append(f"完成处理阶段 {i}")
    
    return state

def verbose_validation_node(state: StreamingState) -> StreamingState:
    """详细输出的验证节点"""
    state["current_step"] = "开始验证"
    time.sleep(0.5)
    
    state["current_step"] = "验证完成"
    state["progress"] = 100
    state["messages"].append("验证通过")
    state["results"]["validation"] = "成功"
    
    return state

# 创建新的流式图
verbose_workflow = StateGraph(StreamingState)
verbose_workflow.add_node("process", verbose_processing_node)
verbose_workflow.add_node("validate", verbose_validation_node)

verbose_workflow.add_edge(START, "process")
verbose_workflow.add_edge("process", "validate")
verbose_workflow.add_edge("validate", END)

verbose_app = verbose_workflow.compile()

# 使用stream()方法获取流式输出
initial_state = {
    "messages": [],
    "current_step": "初始化",
    "progress": 0,
    "results": {}
}

print("=== 流式输出演示 ===")
for chunk in verbose_app.stream(initial_state):
    # chunk是一个字典，包含节点名和该节点的输出状态
    for node_name, state_update in chunk.items():
        print(f"节点: {node_name}")
        print(f"当前步骤: {state_update.get('current_step', 'N/A')}")
        print(f"进度: {state_update.get('progress', 0)}%")
        print(f"消息数量: {len(state_update.get('messages', []))}")
        print("-" * 30)

## 2. 异步流式处理

### 2.1 异步节点定义

In [ ]:
import aiohttp
import asyncio

class AsyncStreamingState(TypedDict):
    messages: List[str]
    current_step: str
    progress: int
    results: Dict[str, Any]
    errors: List[str]

async def async_fetch_data_node(state: AsyncStreamingState) -> AsyncStreamingState:
    """异步数据获取节点"""
    state["current_step"] = "开始获取数据"
    state["messages"].append("启动数据获取任务")
    
    # 模拟多个异步任务
    tasks = []
    for i in range(3):
        tasks.append(fetch_mock_data(i))
    
    # 并行执行任务
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # 处理结果
    successful_results = []
    for i, result in enumerate(results):
        if isinstance(result, Exception):
            state["errors"].append(f"任务 {i} 失败: {str(result)}")
        else:
            successful_results.append(result)
        
        # 更新进度
        state["progress"] = ((i + 1) / len(results)) * 50
        state["current_step"] = f"完成数据获取 {i + 1}/{len(results)}"
    
    state["results"]["fetched_data"] = successful_results
    state["messages"].append(f"数据获取完成，成功: {len(successful_results)}, 失败: {len(state['errors'])}")
    
    return state

async def fetch_mock_data(task_id: int) -> Dict[str, Any]:
    """模拟异步数据获取"""
    await asyncio.sleep(0.5 + task_id * 0.2)  # 不同的延迟
    
    # 模拟偶发错误
    if task_id == 1:
        raise Exception("模拟网络错误")
    
    return {
        "task_id": task_id,
        "data": f"数据_{task_id}",
        "timestamp": datetime.now().isoformat()
    }

async def async_process_data_node(state: AsyncStreamingState) -> AsyncStreamingState:
    """异步数据处理节点"""
    state["current_step"] = "开始处理数据"
    
    fetched_data = state["results"].get("fetched_data", [])
    
    # 异步处理每个数据项
    processed_data = []
    for i, data_item in enumerate(fetched_data):
        await asyncio.sleep(0.2)  # 模拟处理时间
        
        processed_item = {
            "original": data_item,
            "processed": f"已处理_{data_item['task_id']}",
            "process_time": datetime.now().isoformat()
        }
        processed_data.append(processed_item)
        
        # 更新进度
        progress = 50 + ((i + 1) / len(fetched_data)) * 50
        state["progress"] = int(progress)
        state["current_step"] = f"处理数据项 {i + 1}/{len(fetched_data)}"
    
    state["results"]["processed_data"] = processed_data
    state["messages"].append(f"数据处理完成，处理了 {len(processed_data)} 个数据项")
    
    return state

# 创建异步流式图
async_workflow = StateGraph(AsyncStreamingState)
async_workflow.add_node("fetch", async_fetch_data_node)
async_workflow.add_node("process", async_process_data_node)

async_workflow.add_edge(START, "fetch")
async_workflow.add_edge("fetch", "process")
async_workflow.add_edge("process", END)

async_app = async_workflow.compile()

### 2.2 执行异步流式处理

In [ ]:
async def run_async_streaming():
    """运行异步流式处理"""
    initial_state = {
        "messages": [],
        "current_step": "初始化",
        "progress": 0,
        "results": {},
        "errors": []
    }
    
    print("=== 异步流式处理开始 ===")
    
    # 使用astream()进行异步流式处理
    async for chunk in async_app.astream(initial_state):
        for node_name, state_update in chunk.items():
            print(f"\n[{datetime.now().strftime('%H:%M:%S')}] 节点: {node_name}")
            print(f"步骤: {state_update.get('current_step', 'N/A')}")
            print(f"进度: {state_update.get('progress', 0)}%")
            
            if state_update.get('messages'):
                print(f"最新消息: {state_update['messages'][-1]}")
            
            if state_update.get('errors'):
                print(f"错误数量: {len(state_update['errors'])}")
    
    print("\n=== 异步流式处理完成 ===")

# 运行异步示例
await run_async_streaming()

## 3. 事件驱动的实时系统

### 3.1 事件监听器

In [ ]:
from typing import Callable, Any
from collections import defaultdict

class EventSystem:
    """简单的事件系统"""
    
    def __init__(self):
        self.listeners: Dict[str, List[Callable]] = defaultdict(list)
        self.event_history: List[Dict[str, Any]] = []
    
    def on(self, event_type: str, callback: Callable):
        """注册事件监听器"""
        self.listeners[event_type].append(callback)
    
    def emit(self, event_type: str, data: Any = None):
        """触发事件"""
        event = {
            "type": event_type,
            "data": data,
            "timestamp": datetime.now().isoformat()
        }
        
        self.event_history.append(event)
        
        # 调用所有监听器
        for callback in self.listeners[event_type]:
            try:
                callback(event)
            except Exception as e:
                print(f"事件处理错误: {e}")
    
    def get_history(self, event_type: str = None) -> List[Dict[str, Any]]:
        """获取事件历史"""
        if event_type:
            return [e for e in self.event_history if e["type"] == event_type]
        return self.event_history.copy()

# 创建全局事件系统
event_system = EventSystem()

class EventDrivenState(TypedDict):
    messages: List[str]
    current_step: str
    progress: int
    results: Dict[str, Any]
    events: List[Dict[str, Any]]

def event_aware_node(node_name: str):
    """创建事件感知的节点装饰器"""
    def decorator(func):
        def wrapper(state: EventDrivenState) -> EventDrivenState:
            # 触发节点开始事件
            event_system.emit("node_start", {
                "node_name": node_name,
                "input_state": state.copy()
            })
            
            try:
                # 执行节点函数
                result_state = func(state)
                
                # 触发节点完成事件
                event_system.emit("node_complete", {
                    "node_name": node_name,
                    "output_state": result_state.copy()
                })
                
                return result_state
                
            except Exception as e:
                # 触发节点错误事件
                event_system.emit("node_error", {
                    "node_name": node_name,
                    "error": str(e),
                    "state": state.copy()
                })
                raise
        
        return wrapper
    return decorator

@event_aware_node("数据收集")
def data_collection_node(state: EventDrivenState) -> EventDrivenState:
    """数据收集节点"""
    state["current_step"] = "收集数据"
    
    # 模拟数据收集过程
    collected_data = []
    for i in range(1, 6):
        time.sleep(0.2)
        data_item = f"数据项_{i}"
        collected_data.append(data_item)
        
        # 触发数据收集事件
        event_system.emit("data_collected", {
            "item": data_item,
            "count": len(collected_data)
        })
        
        state["progress"] = i * 20
    
    state["results"]["collected_data"] = collected_data
    state["messages"].append(f"收集了 {len(collected_data)} 个数据项")
    
    return state

@event_aware_node("数据分析")
def data_analysis_node(state: EventDrivenState) -> EventDrivenState:
    """数据分析节点"""
    state["current_step"] = "分析数据"
    
    collected_data = state["results"].get("collected_data", [])
    
    # 分析数据
    analysis_results = {
        "total_items": len(collected_data),
        "analysis_time": datetime.now().isoformat(),
        "summary": "数据分析完成"
    }
    
    time.sleep(0.5)
    
    # 触发分析完成事件
    event_system.emit("analysis_complete", {
        "results": analysis_results
    })
    
    state["results"]["analysis"] = analysis_results
    state["messages"].append("数据分析完成")
    state["progress"] = 100
    
    return state

# 注册事件监听器
def on_node_start(event):
    print(f"🟡 节点开始: {event['data']['node_name']} at {event['timestamp']}")

def on_node_complete(event):
    print(f"🟢 节点完成: {event['data']['node_name']} at {event['timestamp']}")

def on_data_collected(event):
    print(f"📊 收集数据: {event['data']['item']} (总计: {event['data']['count']})")

def on_analysis_complete(event):
    print(f"🔍 分析完成: {event['data']['results']['summary']}")

def on_node_error(event):
    print(f"❌ 节点错误: {event['data']['node_name']} - {event['data']['error']}")

# 注册监听器
event_system.on("node_start", on_node_start)
event_system.on("node_complete", on_node_complete)
event_system.on("data_collected", on_data_collected)
event_system.on("analysis_complete", on_analysis_complete)
event_system.on("node_error", on_node_error)

### 3.2 运行事件驱动系统

In [ ]:
# 创建事件驱动的图
event_workflow = StateGraph(EventDrivenState)
event_workflow.add_node("collect", data_collection_node)
event_workflow.add_node("analyze", data_analysis_node)

event_workflow.add_edge(START, "collect")
event_workflow.add_edge("collect", "analyze")
event_workflow.add_edge("analyze", END)

event_app = event_workflow.compile()

# 运行事件驱动系统
initial_state = {
    "messages": [],
    "current_step": "初始化",
    "progress": 0,
    "results": {},
    "events": []
}

print("=== 事件驱动系统演示 ===")
final_state = event_app.invoke(initial_state)

print("\n=== 事件历史 ===")
for event in event_system.get_history():
    print(f"{event['timestamp']}: {event['type']} - {event.get('data', {})}")

print(f"\n=== 最终结果 ===")
print(f"消息: {final_state['messages']}")
print(f"结果: {final_state['results']}")

## 4. WebSocket实时通信系统

### 4.1 WebSocket服务器模拟

In [ ]:
import queue
import threading
from typing import Optional

class MockWebSocketServer:
    """模拟WebSocket服务器"""
    
    def __init__(self):
        self.clients = []
        self.message_queue = queue.Queue()
        self.running = False
    
    def add_client(self, client_id: str):
        """添加客户端"""
        self.clients.append(client_id)
        print(f"客户端 {client_id} 连接")
    
    def remove_client(self, client_id: str):
        """移除客户端"""
        if client_id in self.clients:
            self.clients.remove(client_id)
            print(f"客户端 {client_id} 断开连接")
    
    def broadcast(self, message: Dict[str, Any]):
        """广播消息给所有客户端"""
        for client in self.clients:
            print(f"📡 发送给 {client}: {json.dumps(message, ensure_ascii=False)}")
    
    def send_to_client(self, client_id: str, message: Dict[str, Any]):
        """发送消息给特定客户端"""
        if client_id in self.clients:
            print(f"📨 发送给 {client_id}: {json.dumps(message, ensure_ascii=False)}")
        else:
            print(f"⚠️ 客户端 {client_id} 不存在")

# 创建WebSocket服务器实例
ws_server = MockWebSocketServer()

class WebSocketState(TypedDict):
    messages: List[str]
    current_step: str
    progress: int
    results: Dict[str, Any]
    client_id: str

def websocket_data_processor(state: WebSocketState) -> WebSocketState:
    """WebSocket数据处理节点"""
    client_id = state["client_id"]
    
    # 发送开始消息
    ws_server.send_to_client(client_id, {
        "type": "processing_start",
        "message": "开始数据处理",
        "timestamp": datetime.now().isoformat()
    })
    
    # 模拟数据处理过程
    for i in range(1, 6):
        time.sleep(0.5)
        progress = i * 20
        
        # 发送进度更新
        ws_server.send_to_client(client_id, {
            "type": "progress_update",
            "progress": progress,
            "step": f"处理步骤 {i}/5",
            "timestamp": datetime.now().isoformat()
        })
        
        state["progress"] = progress
        state["current_step"] = f"处理步骤 {i}/5"
    
    # 发送完成消息
    result_data = {"processed_items": 100, "status": "完成"}
    ws_server.send_to_client(client_id, {
        "type": "processing_complete",
        "result": result_data,
        "message": "数据处理完成",
        "timestamp": datetime.now().isoformat()
    })
    
    state["results"]["processing"] = result_data
    state["messages"].append("数据处理完成")
    
    return state

def websocket_analyzer(state: WebSocketState) -> WebSocketState:
    """WebSocket分析节点"""
    client_id = state["client_id"]
    
    # 发送分析开始消息
    ws_server.send_to_client(client_id, {
        "type": "analysis_start",
        "message": "开始数据分析",
        "timestamp": datetime.now().isoformat()
    })
    
    time.sleep(1)
    
    # 分析结果
    analysis_result = {
        "patterns_found": 3,
        "accuracy": 0.95,
        "insights": ["趋势1", "趋势2", "趋势3"]
    }
    
    # 发送分析完成消息
    ws_server.send_to_client(client_id, {
        "type": "analysis_complete",
        "result": analysis_result,
        "message": "数据分析完成",
        "timestamp": datetime.now().isoformat()
    })
    
    state["results"]["analysis"] = analysis_result
    state["messages"].append("数据分析完成")
    state["progress"] = 100
    
    return state

### 4.2 WebSocket实时处理系统

In [ ]:
# 创建WebSocket处理图
ws_workflow = StateGraph(WebSocketState)
ws_workflow.add_node("process", websocket_data_processor)
ws_workflow.add_node("analyze", websocket_analyzer)

ws_workflow.add_edge(START, "process")
ws_workflow.add_edge("process", "analyze")
ws_workflow.add_edge("analyze", END)

ws_app = ws_workflow.compile()

def handle_client_request(client_id: str, request_data: Dict[str, Any]):
    """处理客户端请求"""
    print(f"\n=== 处理客户端 {client_id} 的请求 ===")
    
    # 添加客户端
    ws_server.add_client(client_id)
    
    # 创建初始状态
    initial_state = {
        "messages": [],
        "current_step": "初始化",
        "progress": 0,
        "results": {},
        "client_id": client_id
    }
    
    try:
        # 发送确认消息
        ws_server.send_to_client(client_id, {
            "type": "request_received",
            "message": "请求已接收，开始处理",
            "request_data": request_data,
            "timestamp": datetime.now().isoformat()
        })
        
        # 执行处理流程
        final_state = ws_app.invoke(initial_state)
        
        # 发送最终结果
        ws_server.send_to_client(client_id, {
            "type": "task_complete",
            "message": "任务完成",
            "final_results": final_state["results"],
            "timestamp": datetime.now().isoformat()
        })
        
    except Exception as e:
        # 发送错误消息
        ws_server.send_to_client(client_id, {
            "type": "error",
            "message": f"处理错误: {str(e)}",
            "timestamp": datetime.now().isoformat()
        })
    
    finally:
        # 移除客户端
        ws_server.remove_client(client_id)

# 模拟多个客户端请求
client_requests = [
    {"client_id": "client_001", "data": {"task": "数据分析", "priority": "high"}},
    {"client_id": "client_002", "data": {"task": "数据处理", "priority": "normal"}}
]

# 处理客户端请求
for request in client_requests:
    handle_client_request(request["client_id"], request["data"])
    print("\n" + "="*50 + "\n")

## 5. 实际应用：实时监控仪表板

### 5.1 监控系统状态

In [ ]:
import random
from dataclasses import dataclass, asdict
from typing import List, Dict, Any

@dataclass
class SystemMetrics:
    """系统指标"""
    timestamp: str
    cpu_usage: float
    memory_usage: float
    disk_usage: float
    network_io: float
    active_connections: int
    error_rate: float

class MonitoringDashboard:
    """监控仪表板"""
    
    def __init__(self):
        self.connected_clients = set()
        self.metrics_history = []
        self.alert_thresholds = {
            "cpu_usage": 80.0,
            "memory_usage": 85.0,
            "disk_usage": 90.0,
            "error_rate": 5.0
        }
    
    def add_client(self, client_id: str):
        """添加监控客户端"""
        self.connected_clients.add(client_id)
        print(f"🟢 监控客户端 {client_id} 已连接")
    
    def remove_client(self, client_id: str):
        """移除监控客户端"""
        self.connected_clients.discard(client_id)
        print(f"🔴 监控客户端 {client_id} 已断开")
    
    def generate_metrics(self) -> SystemMetrics:
        """生成系统指标"""
        return SystemMetrics(
            timestamp=datetime.now().isoformat(),
            cpu_usage=random.uniform(20, 95),
            memory_usage=random.uniform(30, 90),
            disk_usage=random.uniform(40, 85),
            network_io=random.uniform(0, 100),
            active_connections=random.randint(10, 500),
            error_rate=random.uniform(0, 10)
        )
    
    def check_alerts(self, metrics: SystemMetrics) -> List[Dict[str, Any]]:
        """检查告警"""
        alerts = []
        
        for metric_name, threshold in self.alert_thresholds.items():
            value = getattr(metrics, metric_name)
            if value > threshold:
                alerts.append({
                    "type": "threshold_exceeded",
                    "metric": metric_name,
                    "value": value,
                    "threshold": threshold,
                    "severity": "high" if value > threshold * 1.1 else "medium",
                    "timestamp": metrics.timestamp
                })
        
        return alerts
    
    def broadcast_metrics(self, metrics: SystemMetrics, alerts: List[Dict[str, Any]]):
        """广播指标给所有客户端"""
        message = {
            "type": "metrics_update",
            "metrics": asdict(metrics),
            "alerts": alerts,
            "client_count": len(self.connected_clients)
        }
        
        for client_id in self.connected_clients:
            print(f"📊 发送指标给 {client_id}:")
            print(f"   CPU: {metrics.cpu_usage:.1f}% | 内存: {metrics.memory_usage:.1f}% | 磁盘: {metrics.disk_usage:.1f}%")
            print(f"   网络IO: {metrics.network_io:.1f} MB/s | 连接: {metrics.active_connections} | 错误率: {metrics.error_rate:.2f}%")
            
            if alerts:
                print(f"   ⚠️ 告警数量: {len(alerts)}")
                for alert in alerts:
                    print(f"      {alert['severity'].upper()}: {alert['metric']} = {alert['value']:.1f} (阈值: {alert['threshold']})")
            print()

# 创建监控仪表板
dashboard = MonitoringDashboard()

class MonitoringState(TypedDict):
    dashboard: MonitoringDashboard
    current_metrics: Optional[SystemMetrics]
    alerts: List[Dict[str, Any]]
    running: bool

def metrics_collector_node(state: MonitoringState) -> MonitoringState:
    """指标收集节点"""
    dashboard = state["dashboard"]
    
    # 生成新的指标
    metrics = dashboard.generate_metrics()
    dashboard.metrics_history.append(metrics)
    
    # 检查告警
    alerts = dashboard.check_alerts(metrics)
    
    # 更新状态
    state["current_metrics"] = metrics
    state["alerts"] = alerts
    
    return state

def metrics_broadcaster_node(state: MonitoringState) -> MonitoringState:
    """指标广播节点"""
    dashboard = state["dashboard"]
    metrics = state["current_metrics"]
    alerts = state["alerts"]
    
    if metrics and dashboard.connected_clients:
        dashboard.broadcast_metrics(metrics, alerts)
    
    return state

def should_continue(state: MonitoringState) -> str:
    """判断是否继续监控"""
    return "collect" if state["running"] else END

# 创建监控图
monitoring_workflow = StateGraph(MonitoringState)
monitoring_workflow.add_node("collect", metrics_collector_node)
monitoring_workflow.add_node("broadcast", metrics_broadcaster_node)

monitoring_workflow.add_edge(START, "collect")
monitoring_workflow.add_edge("collect", "broadcast")
monitoring_workflow.add_conditional_edges(
    "broadcast",
    should_continue,
    {"collect": "collect", END: END}
)

monitoring_app = monitoring_workflow.compile()

### 5.2 运行实时监控系统

In [ ]:
async def run_monitoring_system():
    """运行监控系统"""
    # 添加客户端
    dashboard.add_client("admin_dashboard")
    dashboard.add_client("mobile_app")
    dashboard.add_client("alert_system")
    
    # 创建初始状态
    initial_state = {
        "dashboard": dashboard,
        "current_metrics": None,
        "alerts": [],
        "running": True
    }
    
    print("=== 启动实时监控系统 ===")
    print(f"连接的客户端: {list(dashboard.connected_clients)}")
    print()
    
    # 运行5个监控周期
    for cycle in range(5):
        print(f"--- 监控周期 {cycle + 1} ---")
        
        # 执行一次监控循环
        result_state = monitoring_app.invoke(initial_state)
        
        # 更新运行状态
        initial_state = result_state
        
        # 等待下一个周期
        await asyncio.sleep(2)
    
    # 停止监控
    initial_state["running"] = False
    print("\n=== 停止监控系统 ===")
    
    # 移除客户端
    for client_id in list(dashboard.connected_clients):
        dashboard.remove_client(client_id)
    
    print(f"\n总共收集了 {len(dashboard.metrics_history)} 个指标数据点")
    
    # 显示历史指标统计
    if dashboard.metrics_history:
        avg_cpu = sum(m.cpu_usage for m in dashboard.metrics_history) / len(dashboard.metrics_history)
        avg_memory = sum(m.memory_usage for m in dashboard.metrics_history) / len(dashboard.metrics_history)
        max_connections = max(m.active_connections for m in dashboard.metrics_history)
        
        print(f"平均CPU使用率: {avg_cpu:.1f}%")
        print(f"平均内存使用率: {avg_memory:.1f}%")
        print(f"最大连接数: {max_connections}")

# 运行监控系统
await run_monitoring_system()

## 6. 实践练习

### 练习1：创建股票价格实时推送系统
创建一个模拟股票价格实时推送系统，要求：
- 模拟多只股票的价格变动
- 支持客户端订阅特定股票
- 实时推送价格变动和告警信息
- 支持历史数据查询

### 练习2：实现聊天室系统
使用流式输出实现一个聊天室系统，要求：
- 支持多用户实时聊天
- 消息广播和私聊功能
- 用户状态管理（在线/离线）
- 消息历史记录

### 练习3：构建实时日志分析系统
创建一个实时日志分析系统，要求：
- 实时处理日志流
- 异常检测和告警
- 统计分析和可视化
- 支持多种日志格式

## 7. 总结

本教程详细介绍了LangGraph的流式输出和实时交互功能：

### 关键概念
1. **流式输出**：使用`stream()`和`astream()`方法获取执行过程中的中间结果
2. **异步处理**：支持异步节点和并发执行，提高系统性能
3. **事件系统**：通过事件监听实现松耦合的系统组件通信
4. **实时通信**：模拟WebSocket等实时通信协议
5. **监控仪表板**：实现实时指标收集和广播系统

### 最佳实践
1. **合理使用流式输出**：对于长时间运行的任务，使用流式输出提供进度反馈
2. **异步优化**：对于I/O密集型任务，使用异步节点提高并发性能
3. **事件解耦**：通过事件系统实现组件间的松耦合
4. **错误处理**：在实时系统中实现robust的错误处理和恢复机制
5. **性能监控**：实时监控系统性能和资源使用情况

### 应用场景
- 实时数据处理管道
- 监控和告警系统
- 实时通信应用
- 在线协作平台
- 实时分析仪表板

通过掌握这些流式输出和实时交互技术，你可以构建高效、响应迅速的实时应用系统。